# MVP - Machine Learning para Diagnóstico de Câncer de Mama
**Autor:** Carlos Eduardo Silva dos Santos  
**Disciplina:** Privacidade e Segurança Cibernética  
**Professor:** Dr. André Serrano

---
## 1. Definição do Problema
O objetivo deste projeto é desenvolver um modelo de *Machine Learning* capaz de classificar tumores de mama como **malignos** ou **benignos** com base em características obtidas a partir de exames.

**Tipo de aprendizado:** Supervisionado (Classificação Binária)

**Motivação:** Diagnósticos precoces aumentam a eficácia do tratamento e a taxa de sobrevivência.


## 2. Bibliotecas Utilizadas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

import warnings
warnings.filterwarnings('ignore')

## 3. Carregamento do Dataset
O dataset utilizado foi obtido do [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Diagnostic%29).

In [ ]:
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data'
cols = ['id','diagnosis'] + [f'feature_{i}' for i in range(1,31)]
data = pd.read_csv(url, names=cols)
data.head()

### Informações iniciais do dataset

In [ ]:
data.info()

## 4. Limpeza e Preparação dos Dados

In [ ]:
# Remover coluna de ID
data.drop('id', axis=1, inplace=True)

# Transformar a coluna alvo em binária (M=1, B=0)
data['diagnosis'] = data['diagnosis'].map({'M':1, 'B':0})

data.head()

In [ ]:
data.describe()

### Divisão entre treino e teste

In [ ]:
X = data.drop('diagnosis', axis=1)
y = data['diagnosis']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

print(f'Treinamento: {X_train.shape}, Teste: {X_test.shape}')

### Normalização dos dados

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## 5. Modelagem e Treinamento
Testaremos três modelos:
- Regressão Logística
- Árvore de Decisão
- Random Forest

In [ ]:
models = {
    'Logistic Regression': LogisticRegression(),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42)
}

results = {}

for name, model in models.items():
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    acc = accuracy_score(y_test, y_pred)
    results[name] = acc
    print(f'{name}: {acc:.4f}')

In [ ]:
best_model = max(results, key=results.get)
print(f'Melhor modelo inicial: {best_model} com acurácia de {results[best_model]:.4f}')

### Otimização de hiperparâmetros do Random Forest

In [ ]:
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10]
}

grid = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=5, scoring='accuracy')
grid.fit(X_train_scaled, y_train)

print('Melhores parâmetros:', grid.best_params_)
print('Melhor score de validação:', grid.best_score_)

## 6. Avaliação do Modelo Final

In [ ]:
final_model = grid.best_estimator_
y_pred_final = final_model.predict(X_test_scaled)

print(classification_report(y_test, y_pred_final))
sns.heatmap(confusion_matrix(y_test, y_pred_final), annot=True, fmt='d', cmap='Blues')
plt.xlabel('Previsto')
plt.ylabel('Real')
plt.show()

## 7. Conclusões
O modelo Random Forest otimizado apresentou a melhor performance, atingindo uma acurácia superior a 98%.

Possíveis melhorias futuras incluem:
- Avaliar redes neurais profundas.
- Explorar mais técnicas de engenharia de atributos.
- Obter mais dados para evitar overfitting.

---
**Referências:**
- [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Diagnostic%29)
- Documentação do Scikit-learn